## 파일 업로드/불러오기 ##

In [ ]:
# 버전 명시

import sys
import pandas as pd
import numpy as np

print("Python version:", sys.version)
print("pandas version:", pd.__version__)
print("numpy version:", np.__version__)

In [ ]:
import pandas as pd

# 데이터셋 불러오기
try:
    df = pd.read_csv('data/Health_2023.csv', encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv('data/Health_2023.csv', encoding='cp949')

# 불필요한 컬럼 삭제
df = df.drop(['결손치 유무', '치아마모증유무', '제3대구치(사랑니) 이상'], axis=1)

# '총콜레스테롤' 컬럼 기준 결측값 삭제
df = df.dropna(subset=['총콜레스테롤'])

df.to_csv('data/health_2023_cleaned.csv', index=False)

In [ ]:
df = pd.read_csv('data/health_2023_cleaned.csv')
df.info()
df.head()

In [ ]:
import matplotlib.pyplot as plt

# 한글 폰트 설정 (macOS: AppleGothic)
plt.rc('font', family='AppleGothic')
plt.rc('axes', unicode_minus=False)

# 수치형 컬럼만 선택
numeric_cols = df.columns

# 각 컬럼별로 박스플롯 생성 및 저장
for col in numeric_cols:
    plt.figure(figsize=(8, 4))
    df.boxplot(column=col)
    plt.title(f'Boxplot of {col} (Outlier Visualization)')
    plt.tight_layout()
    plt.savefig(f'plots/boxplot_{col}.png')
    plt.close()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 설정 (macOS: AppleGothic)
plt.rc('font', family='AppleGothic')
plt.rc('axes', unicode_minus=False)

# 수치형 컬럼만 선택
df_numeric = df.select_dtypes(include=['float64', 'int64'])

# 각 컬럼별로 분포 시각화 및 저장
for col in df_numeric.columns:
    plt.figure(figsize=(8, 4))
    sns.histplot(df_numeric[col].dropna(), kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('빈도')
    plt.tight_layout()
    plt.savefig(f'plots/distribution_{col}.png')
    plt.close()

## 이상치 처리 과정 ##

In [ ]:
columns_to_show = ['감마지티피']  # 원하는 컬럼 추가
high_bp_top20 = df.nlargest(50, '감마지티피')[columns_to_show]
print(high_bp_top20)

In [ ]:
# 말이 안 되는 값 1차 제거 (print된 위 코드 확인 및 boxplot 확인 후)

df = df[df['감마지티피'] != 9999.0]

In [ ]:
# 아래 기준은 정상 범주가 아니라, 생리학적으로 가능하다고 판단되는 임상적 범주로 설정

col = '감마지티피'
lower_bound = 1
upper_bound = 3000

outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
print(f"{col} 이상치 개수:", len(outliers))
print(lower_bound, upper_bound)
print(outliers[[col]])

In [ ]:
# 임상적 기준으로 탐지된 감마지티피 이상치 삭제
df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

In [ ]:
# 아래 기준은 정상 범주가 아니라, 생리학적으로 가능하다고 판단되는 임상적 범주로 설정
# 이상치 삭제 확인

col = '감마지티피'
lower_bound = 1
upper_bound = 3000

outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
print(f"{col} 이상치 개수:", len(outliers))
print(lower_bound, upper_bound)
print(outliers[[col]])

In [ ]:
columns_to_show = ['감마지티피']
high_bp_top20 = df.nlargest(20, '감마지티피')[columns_to_show]
print(high_bp_top20)

In [ ]:
columns_to_show = ['수축기혈압', '이완기혈압']  # 원하는 컬럼 추가
high_bp_top10 = df.nlargest(10, '수축기혈압')[columns_to_show]
print(high_bp_top10)

In [ ]:
# 아래 기준은 정상 범주가 아니라, 생리학적으로 가능하다고 판단되는 임상적 범주로 설정

col = '수축기혈압'
lower_bound = 40
upper_bound = 300

outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
print(f"{col} 이상치 개수:", len(outliers))
print(lower_bound, upper_bound)
print(outliers[[col]])

In [ ]:
# 아래 기준은 정상 범주가 아니라, 생리학적으로 가능하다고 판단되는 범주로 설정

col = '수축기혈압'
lower_bound = 70
upper_bound = 250

outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
print(f"{col} 이상치 개수:", len(outliers))
print(lower_bound, upper_bound)
print(outliers[[col]])

In [ ]:
# 임상적 범주로 탐지된 수축기혈압 이상치 삭제
df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

In [ ]:
columns_to_show = ['수축기혈압', '이완기혈압']  # 원하는 컬럼 추가
high_bp_top10 = df.nlargest(10, '수축기혈압')[columns_to_show]
print(high_bp_top10)

In [ ]:
columns_to_show = ['시력(우)', '시력(좌)']  # 원하는 컬럼 추가
high_bp_top20 = df.nlargest(20, '시력(우)')[columns_to_show]
print(high_bp_top20)

In [ ]:
# 시력 9.9값들은 실명으로 간주하여 0으로 변경 (한글 파일 참조)

df['시력(우)'] = df['시력(우)'].replace(9.9, 0)
df['시력(좌)'] = df['시력(좌)'].replace(9.9, 0)

In [ ]:
columns_to_show = ['식전혈당(공복혈당)']  # 원하는 컬럼 추가
high_bp_top20 = df.nlargest(20, '식전혈당(공복혈당)')[columns_to_show]
print(high_bp_top20)

In [ ]:
# 아래 기준은 정상 범주가 아니라, 생리학적으로 가능하다고 판단되는 범주로 설정

col = '식전혈당(공복혈당)'
lower_bound = 20
upper_bound = 800

outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
print(f"{col} 이상치 개수:", len(outliers))
print(lower_bound, upper_bound)
print(outliers[[col]])

In [ ]:
# 임상적 범주로 탐지된 수축기혈압 이상치 삭제
df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

In [ ]:
# 아래 기준은 정상 범주가 아니라, 생리학적으로 가능하다고 판단되는 범주로 설정

col = '이완기혈압'
lower_bound = 20
upper_bound = 160

outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
print(f"{col} 이상치 개수:", len(outliers))
print(lower_bound, upper_bound)
print(outliers[[col]])

In [ ]:
# 임상적 범주로 탐지된 수축기혈압 이상치 삭제
df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

In [ ]:
# 아래 기준은 정상 범주가 아니라, 생리학적으로 가능하다고 판단되는 범주로 설정

col = '총콜레스테롤'
lower_bound = 0
upper_bound = 800

outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
print(f"{col} 이상치 개수:", len(outliers))
print(lower_bound, upper_bound)
print(outliers[[col]])

In [ ]:
# 임상적 범주로 탐지된 수축기혈압 이상치 삭제
df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

In [ ]:
# 아래 기준은 정상 범주가 아니라, 생리학적으로 가능하다고 판단되는 범주로 설정

col = '트리글리세라이드'
lower_bound = 0
upper_bound = 5000

outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
print(f"{col} 이상치 개수:", len(outliers))
print(lower_bound, upper_bound)
print(outliers[[col]])

In [ ]:
# 아래 기준은 정상 범주가 아니라, 생리학적으로 가능하다고 판단되는 범주로 설정

col = '허리둘레'
lower_bound = 0
upper_bound = 200

outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
print(f"{col} 이상치 개수:", len(outliers))
print(lower_bound, upper_bound)
print(outliers[[col]])

In [ ]:
# 임상적 범주로 탐지된 수축기혈압 이상치 삭제
df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

In [ ]:
# 아래 기준은 정상 범주가 아니라, 생리학적으로 가능하다고 판단되는 범주로 설정

col = '혈색소'
lower_bound = 2
upper_bound = 25

outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
print(f"{col} 이상치 개수:", len(outliers))
print(lower_bound, upper_bound)
print(outliers[[col]])

In [ ]:
# 임상적 범주로 탐지된 수축기혈압 이상치 삭제
df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

In [ ]:
# 아래 기준은 정상 범주가 아니라, 생리학적으로 가능하다고 판단되는 범주로 설정

col = '혈청지오티(AST)'
lower_bound = 0
upper_bound = 5000

outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
print(f"{col} 이상치 개수:", len(outliers))
print(lower_bound, upper_bound)
print(outliers[[col]])

In [ ]:
# 아래 기준은 정상 범주가 아니라, 생리학적으로 가능하다고 판단되는 범주로 설정

col = '혈청지피티(ALT)'
lower_bound = 0
upper_bound = 5000

outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
print(f"{col} 이상치 개수:", len(outliers))
print(lower_bound, upper_bound)
print(outliers[[col]])

In [ ]:
# 아래 기준은 정상 범주가 아니라, 생리학적으로 가능하다고 판단되는 범주로 설정

col = '혈청크레아티닌'
lower_bound = 0
upper_bound = 20

outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
print(f"{col} 이상치 개수:", len(outliers))
print(lower_bound, upper_bound)
print(outliers[[col]])

In [ ]:
# 임상적 범주로 탐지된 수축기혈압 이상치 삭제
df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

In [ ]:
# 아래 기준은 정상 범주가 아니라, 생리학적으로 가능하다고 판단되는 범주로 설정

col = 'HDL콜레스테롤'
lower_bound = 0
upper_bound = 300

outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
print(f"{col} 이상치 개수:", len(outliers))
print(lower_bound, upper_bound)
print(outliers[[col]])

In [ ]:
# 임상적 범주로 탐지된 수축기혈압 이상치 삭제
df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

In [ ]:
# 아래 기준은 정상 범주가 아니라, 생리학적으로 가능하다고 판단되는 범주로 설정

col = 'LDL콜레스테롤'
lower_bound = 0
upper_bound = 600

outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
print(f"{col} 이상치 개수:", len(outliers))
print(lower_bound, upper_bound)
print(outliers[[col]])

In [ ]:
# 임상적 범주로 탐지된 수축기혈압 이상치 삭제
df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

## 이상치 처리 방식 ##

1. 의료 데이터셋에서 가장 많이 사용하는 IQR 방식으로 이상치를 처리해보려 하였으나, 본 데이터셋에 적용시키게 되면 정상 범주들의 데이터들이 다수 이상치로 판단되는 오류가 있음
2. 극단적인 값들 또한 분석/예측하는 데에 의미가 있는 데이터라고 판단하여 임상적으로 가능한 값들을 각 컬럼별로 정의하여 이상치 제거 (데이터셋도 최대한 보존 가능)

In [ ]:
# 이상치 제거 후 최종 데이터셋 저장
df.to_csv('data/health_2023_cleaned_final.csv', index=False)